<h1><b>For Linearly Seperable Data</b></h1>

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from perceptron import Perceptron

### Create data set by combining the two class data
def two_class_data(train1, train2, validate1, validate2, test1, test2):
    trainX = np.concatenate((train1, train2), axis=0)
    trainY = np.array([0 for i in range(len(train1))] + [1 for i in range(len(train2))])
    validateX = np.concatenate((validate1, validate2), axis=0)
    validateY = np.array([0 for i in range(len(validate1))] + [1 for i in range(len(validate2))])
    testX = np.concatenate((test1, test2), axis=0)
    testY = np.array([0 for i in range(len(test1))] + [1 for i in range(len(test2))])
    return trainX, trainY, validateX, validateY, testX, testY

### Create data set by combining all three class data
def all_class_data(train1, train2, train3, validate1, validate2, validate3, test1, test2, test3):
    trainX = np.concatenate((train1, train2, train3), axis=0)
    trainY = np.array([0 for i in range(len(train1))] + [1 for i in range(len(train2))] + [2 for i in range(len(train3))])
    validateX = np.concatenate((validate1, validate2, validate3), axis=0)
    validateY = np.array([0 for i in range(len(validate1))] + [1 for i in range(len(validate2))] + [2 for i in range(len(validate3))])
    testX = np.concatenate((test1, test2, test3), axis=0)
    testY = np.array([0 for i in range(len(test1))] + [1 for i in range(len(test2))] + [2 for i in range(len(test3))])
    return trainX, trainY, validateX, validateY, testX, testY

#### Path of all class dataset 
f1 = r"D:\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\LS_Group32\Class1.txt"
f2 = r"D:\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\LS_Group32\Class2.txt"
f3 = r"D:\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\LS_Group32\Class3.txt"

### Devide the Class1 data into training, validation, and testing data
df = pd.read_csv(f1, delimiter=' ', header=None)
train1, validate1, test1 = np.split(df, [int(0.6*len(df)), int(0.8*len(df))])
### Devide the Class2 data into training, validation, and testing data
df = pd.read_csv(f2, delimiter=' ', header=None)
train2, validate2, test2 = np.split(df, [int(0.6*len(df)), int(0.8*len(df))])
### Devide the Class3 data into training, validation, and testing data
df = pd.read_csv(f3, delimiter=' ', header=None)
train3, validate3, test3 = np.split(df, [int(0.6*len(df)), int(0.8*len(df))])

#### For classification between Class1 and Class2 ####
trainX12, trainY12, validateX12, validateY12, testX12, testY12 = two_class_data(train1, train2, validate1, validate2, test1, test2)
trainX13, trainY13, validateX13, validateY13, testX13, testY13 = two_class_data(train1, train3, validate1, validate3, test1, test3)
trainX23, trainY23, validateX23, validateY23, testX23, testY23 = two_class_data(train2, train3, validate2, validate3, test2, test3)
trainX, trainY, validateX, validateY, testX, testY = all_class_data(train1, train2, train3, validate1, validate2, validate3, test1, test2, test3)
        
### Classification between class1 and class2 
p12 = Perceptron(learning_rate=0.001, epoch=1000)
p12.train(trainX12, trainY12, epoch=True)
predictions12 = p12.predict(testX12)

### Classification between class3 and class1 
p13 = Perceptron(learning_rate=0.001, epoch=1000)
p13.train(trainX13, trainY13, epoch=True)
predictions13 = p13.predict(testX13)

### Classification between class2 and class3 
p23 = Perceptron(learning_rate=0.001, epoch=1000)
p23.train(trainX23, trainY23, epoch=True)
predictions23 = p23.predict(testX23)

def predictAllClass(testX, model12, model13, model23):
    final_pred = []
    for i in range(len(testX)):
        x = testX[i:i+1]
        pred12 = model12.predict(x)
        pred13 = model13.predict(x)
        pred23 = model23.predict(x)
        if pred12[0,0] == 0 and pred13[0,0] == 0 and pred23[0,0] == 0:
            final_pred.append(0)
        elif pred12[0,0] == 0 and pred13[0,0] == 0 and pred23[0,0] == 1:
            final_pred.append(0)
        elif pred12[0,0] == 0 and pred13[0,0] == 1 and pred23[0,0] == 1:
            final_pred.append(2)
        elif pred12[0,0] == 1 and pred13[0,0] == 0 and pred23[0,0] == 0:
            final_pred.append(1)
        elif pred12[0,0] == 1 and pred13[0,0] == 1 and pred23[0,0] == 0:
            final_pred.append(1)
        elif pred12[0,0] == 1 and pred13[0,0] == 1 and pred23[0,0] == 1:
            final_pred.append(2)
        else:
            final_pred.append(-1)
    return final_pred

final_pred = predictAllClass(testX, p12, p13, p23)

def decision_boundary_plot():
    x_min, x_max = testX[:, 0].min() - 1, testX[:, 0].max() + 1
    y_min, y_max = testX[:, 1].min() - 1, testX[:, 1].max() + 1
    h=0.05
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    g_data = np.c_[xx.ravel(), yy.ravel()]
    
    predictedLabel = predictAllClass(g_data, p12, p13, p23)
    color = ['#EE6363', '#BCEE68', '#B2DFEE']
    predictedColor = [color[i] for i in predictedLabel]
    
    plt.scatter(g_data[:,0], g_data[:,1], s=5, color=predictedColor)
    plt.scatter(np.array(test1)[:,0], np.array(test1)[:,1], s=5, color='red', label='Class1')
    plt.scatter(np.array(test2)[:,0], np.array(test2)[:,1], s=5, color='green', label='Class2')
    plt.scatter(np.array(test3)[:,0], np.array(test3)[:,1], s=5, color='blue', label='Class3')
    plt.legend(bbox_to_anchor=(0.05, 1.15), loc='upper left', ncol=3)
    plt.xlabel('x-axis')
    plt.ylabel('y-axis')
    # plt.title('Constant density contour plot of Class1, Class2, and Class3 with its dataset')
    plt.savefig("decision_boundry_Perceptron_LS.png", dpi=600, bbox_inches="tight")
    plt.clf()

### Call the 'decision_boundary_plot' function to plot the decision boundary
decision_boundary_plot()

def epochVsError_plot(model, classes):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    error = model.avg_error
    nepoch = [i+1 for i in range(len(error))]
    plt.scatter(nepoch, error, marker='o', s=5, facecolors='b', edgecolors='b')
    plt.xlabel('Number of Epoch')
    plt.ylabel('Average Error')
#     plt.title('Average error vs number of epoch for training of Class{} and Class{}'.format(classes[0], classes[1]))
    plt.savefig("AvgErrorVsEpoch{}{}_Perceptron_LS.png".format(classes[0], classes[1]), dpi=600, bbox_inches="tight")
    plt.clf()

### Call the 'epochVsError_plot' function for all pair of classes
epochVsError_plot(p12, [1, 2])
epochVsError_plot(p13, [1, 3])
epochVsError_plot(p23, [2, 3])

### To calculate the confusion matrix and classification accuracy
def confusion_matrix(actual, predicted):
    cm = np.zeros((3, 3))
    for i, j in zip(actual, predicted):
        cm[i][j] += 1
    ### For classification accuracy
    accuracy = np.sum(actual == predicted) * 100.0 / float(len(actual))
    return cm, accuracy

### Call the confusion_matrix function
CM, Accuracy = confusion_matrix(testY, final_pred)
print("Confusion Matrix: {}".format(CM))
print("Classification Accuracy: {}".format(Accuracy))

Confusion Matrix: [[100.   0.   0.]
 [  0. 100.   0.]
 [  0.   0. 100.]]
Classification Accuracy: 100.0


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<h1><b>For Non Linearly Seperable Data</b></h1>

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from perceptron import Perceptron

### Create data set by combining the two class data
def two_class_data(train1, train2, validate1, validate2, test1, test2):
    trainX = np.concatenate((train1, train2), axis=0)
    trainY = np.array([0 for i in range(len(train1))] + [1 for i in range(len(train2))])
    validateX = np.concatenate((validate1, validate2), axis=0)
    validateY = np.array([0 for i in range(len(validate1))] + [1 for i in range(len(validate2))])
    testX = np.concatenate((test1, test2), axis=0)
    testY = np.array([0 for i in range(len(test1))] + [1 for i in range(len(test2))])
    return trainX, trainY, validateX, validateY, testX, testY

### Create data set by combining all three class data
def all_class_data(train1, train2, train3, validate1, validate2, validate3, test1, test2, test3):
    trainX = np.concatenate((train1, train2, train3), axis=0)
    trainY = np.array([0 for i in range(len(train1))] + [1 for i in range(len(train2))] + [2 for i in range(len(train3))])
    validateX = np.concatenate((validate1, validate2, validate3), axis=0)
    validateY = np.array([0 for i in range(len(validate1))] + [1 for i in range(len(validate2))] + [2 for i in range(len(validate3))])
    testX = np.concatenate((test1, test2, test3), axis=0)
    testY = np.array([0 for i in range(len(test1))] + [1 for i in range(len(test2))] + [2 for i in range(len(test3))])
    return trainX, trainY, validateX, validateY, testX, testY

#### Path of all class dataset 
f1 = r"D:\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\NLS_Group32.txt"

### Devide the Class1 data into training, validation, and testing data
df = pd.read_csv(f1, delimiter=' ', header=None)
df1, df2, df3 = np.split(df, [500, 1000])
train1, validate1, test1 = np.split(df1, [int(0.6*len(df1)), int(0.8*len(df1))])
### Devide the Class2 data into training, validation, and testing data
train2, validate2, test2 = np.split(df2, [int(0.6*len(df2)), int(0.8*len(df2))])
### Devide the Class3 data into training, validation, and testing data
train3, validate3, test3 = np.split(df3, [int(0.6*len(df3)), int(0.8*len(df3))])

#### For classification between Class1 and Class2 ####
trainX12, trainY12, validateX12, validateY12, testX12, testY12 = two_class_data(train1, train2, validate1, validate2, test1, test2)
trainX13, trainY13, validateX13, validateY13, testX13, testY13 = two_class_data(train1, train3, validate1, validate3, test1, test3)
trainX23, trainY23, validateX23, validateY23, testX23, testY23 = two_class_data(train2, train3, validate2, validate3, test2, test3)
trainX, trainY, validateX, validateY, testX, testY = all_class_data(train1, train2, train3, validate1, validate2, validate3, test1, test2, test3)

### Classification between class1 and class2 
p12 = Perceptron(learning_rate=0.001, epoch=1000)
p12.train(trainX12, trainY12, epoch=True)
predictions12 = p12.predict(testX12)

### Classification between class3 and class1 
p13 = Perceptron(learning_rate=0.001, epoch=1000)
p13.train(trainX13, trainY13, epoch=True)
predictions13 = p13.predict(testX13)

### Classification between class2 and class3 
p23 = Perceptron(learning_rate=0.001, epoch=1000)
p23.train(trainX23, trainY23, epoch=True)
predictions23 = p23.predict(testX23)

def predictAllClass(testX, model12, model13, model23):
    final_pred = []
    for i in range(len(testX)):
        x = testX[i:i+1]
        pred12 = model12.predict(x)
        pred13 = model13.predict(x)
        pred23 = model23.predict(x)
        if pred12[0,0] == 0 and pred13[0,0] == 0 and pred23[0,0] == 0:
            final_pred.append(0)
        elif pred12[0,0] == 0 and pred13[0,0] == 0 and pred23[0,0] == 1:
            final_pred.append(0)
        elif pred12[0,0] == 0 and pred13[0,0] == 1 and pred23[0,0] == 1:
            final_pred.append(2)
        elif pred12[0,0] == 1 and pred13[0,0] == 0 and pred23[0,0] == 0:
            final_pred.append(1)
        elif pred12[0,0] == 1 and pred13[0,0] == 1 and pred23[0,0] == 0:
            final_pred.append(1)
        elif pred12[0,0] == 1 and pred13[0,0] == 1 and pred23[0,0] == 1:
            final_pred.append(2)
        else:
            final_pred.append(-1)
    return final_pred

final_pred = predictAllClass(testX, p12, p13, p23)

def decision_boundary_plot():
    x_min, x_max = testX[:, 0].min() - 1, testX[:, 0].max() + 1
    y_min, y_max = testX[:, 1].min() - 1, testX[:, 1].max() + 1
    h=0.005
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    g_data = np.c_[xx.ravel(), yy.ravel()]
    
    predictedLabel = predictAllClass(g_data, p12, p13, p23)
    color = ['#EE6363', '#BCEE68', '#B2DFEE']
    predictedColor = [color[i] for i in predictedLabel]
    
    plt.scatter(g_data[:,0], g_data[:,1], s=5, color=predictedColor)
    plt.scatter(np.array(test1)[:,0], np.array(test1)[:,1], s=5, color='red', label='Class1')
    plt.scatter(np.array(test2)[:,0], np.array(test2)[:,1], s=5, color='green', label='Class2')
    plt.scatter(np.array(test3)[:,0], np.array(test3)[:,1], s=5, color='blue', label='Class3')
    plt.legend(bbox_to_anchor=(0.05, 1.15), loc='upper left', ncol=3)
    plt.xlabel('x-axis')
    plt.ylabel('y-axis')
    # plt.title('Constant density contour plot of Class1, Class2, and Class3 with its dataset')
    plt.savefig("decision_boundry_Perceptron_NLS.png", dpi=600, bbox_inches="tight")
    plt.clf()

### Call the 'decision_boundary_plot' function to plot the decision boundary
decision_boundary_plot()

def epochVsError_plot(model, classes):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    error = model.avg_error
    nepoch = [i+1 for i in range(len(error))]
    plt.scatter(nepoch, error, marker='o', s=5, facecolors='b', edgecolors='b')
    plt.xlabel('Number of Epoch')
    plt.ylabel('Average Error')
#     plt.title('Average error vs number of epoch for training of Class{} and Class{}'.format(classes[0], classes[1]))
    plt.savefig("AvgErrorVsEpoch{}{}_Perceptron_NLS.png".format(classes[0], classes[1]), dpi=600, bbox_inches="tight")
    plt.clf()

### Call the 'epochVsError_plot' function for all pair of classes
epochVsError_plot(p12, [1, 2])
epochVsError_plot(p13, [1, 3])
epochVsError_plot(p23, [2, 3])

### To calculate the confusion matrix and classification accuracy
def confusion_matrix(actual, predicted):
    cm = np.zeros((3, 3))
    for i, j in zip(actual, predicted):
        cm[i][j] += 1
    ### For classification accuracy
    accuracy = np.sum(actual == predicted) * 100.0 / float(len(actual))
    return cm, accuracy

### Call the confusion_matrix function
CM, Accuracy = confusion_matrix(testY, final_pred)
print("Confusion Matrix: {}".format(CM))
print("Classification Accuracy: {}".format(Accuracy))

Confusion Matrix: [[93.  7.  0.]
 [ 5. 92.  3.]
 [ 0.  2. 98.]]
Classification Accuracy: 94.33333333333333


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<h1><b>Univariant Regression using single Perceptron</b></h1>

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from perceptron import Perceptron

#### Path of Univariant regression dataset 
f1 = r"D:\Sujeet_PhD\Course_Work\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Regression\UnivariateData\32.csv"

### Devide the data into training, validation, and testing data
df = pd.read_csv(f1, header=None)
train, validate, test = np.split(df, [int(0.6*len(df)), int(0.8*len(df))])
trainX, trainY = np.array(train[0]).reshape((train.shape[0], 1)), np.array(train[1])
validateX, validateY = np.array(validate[0]).reshape((validate.shape[0], 1)), np.array(validate[1])
testX, testY = np.array(test[0]).reshape((test.shape[0], 1)), np.array(test[1])

### Classification between class1 and class2 
p1 = Perceptron(learning_rate=0.00001, epoch=5000, prediction='regression')
p1.train(trainX, trainY, epoch=True)

predictions_train = p1.predictR(trainX)
predictions_validate = p1.predictR(validateX)
predictions_test = p1.predictR(testX)

def epochVsError_plot(model):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    error = model.avg_error
    nepoch = [i+1 for i in range(len(error))]
    plt.scatter(nepoch, error, marker='o', s=5, facecolors='none', edgecolors='b')
    plt.xlabel('Number of Epoch')
    plt.ylabel('Average Error')
#     plt.title('Average error vs number of epoch for training of univariant regression data')
    plt.savefig("AvgErrorVsEpoch_Perceptron_Regression1.png", dpi=600, bbox_inches="tight")
    plt.clf()
    
def modelAndTarget_plot(target, model, label='train'):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    n_item = [i+1 for i in range(len(target))]
    plt.scatter(n_item, model, marker='o', s=5, facecolors='b', edgecolors='b', label='Model Output')
    plt.scatter(n_item, target, marker='s', s=5, facecolors='r', edgecolors='r', label='Target Output')
    plt.legend(bbox_to_anchor=(0.15, 1.2), loc='upper left', ncol=2)
    plt.xlabel('Dataset')
    plt.ylabel('Regression Output')
#     plt.title('Model output and target output of {} dataset'.format(label))
    plt.savefig("modelAndTarget_{}_Perceptron_Regression1.png".format(label), dpi=600, bbox_inches="tight")
    plt.clf()

def modelVsTarget_plot(target, model, label='train'):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    plt.scatter(target, model, marker='o', s=5, facecolors='b', edgecolors='b')
    plt.xlabel('Target Output')
    plt.ylabel('Model Output')
#     plt.title('Model output vs target output of {} dataset'.format(label))
    plt.savefig("modelVsTarget_{}_Perceptron_Regression1.png".format(label), dpi=600, bbox_inches="tight")
    plt.clf()
    
def MSE(target, model):
    target = np.array(target)
    model = np.array(model)
    mse = np.mean(np.power(target - model, 2))
    return mse

def plot_mse(mse):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    x_data = ['train', 'validation', 'test']
    plt.scatter(x_data, mse, marker='o', s=5, facecolors='b', edgecolors='b')
    plt.xlabel('Type of Dataset')
    plt.ylabel('Mean Square Error')
#     plt.title('Mean squared error (MSE) on training data, validation data and test data')
    plt.savefig("MSE_Perceptron_Regression1.png", dpi=600, bbox_inches="tight")
    plt.clf()
    
### Call the 'epochVsError_plot' function for all pair of classes
epochVsError_plot(p1)
### Model output and target output plot
modelAndTarget_plot(trainY, predictions_train, label='train')
modelAndTarget_plot(testY, predictions_test, label='test')
modelAndTarget_plot(validateY, predictions_validate, label='validation')
### Model output vs target output plot
modelVsTarget_plot(trainY, predictions_train, label='train')
modelVsTarget_plot(testY, predictions_test, label='test')
modelVsTarget_plot(validateY, predictions_validate, label='validation')
### For Mean square Error
mse_train = MSE(trainY, predictions_train)
mse_val = MSE(validateY, predictions_validate)
mse_test = MSE(testY, predictions_test)
plot_mse([mse_train, mse_val, mse_test])


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<h1><b>Multivariant Regression using single Perceptron</b></h1>

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from perceptron import Perceptron

#### Path of Bivariant regression dataset 
f1 = r"D:\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Regression\BivariateData\32.csv"

### Devide the data into training, validation, and testing data
df = pd.read_csv(f1, header=None)
train, validate, test = np.split(df, [int(0.6*len(df)), int(0.8*len(df))])
trainX, trainY = train.to_numpy()[:,0:2].reshape((train.shape[0], 2)), train.to_numpy()[:,2]
validateX, validateY = validate.to_numpy()[:,0:2].reshape((validate.shape[0], 2)), validate.to_numpy()[:,2]
testX, testY = test.to_numpy()[:,0:2].reshape((test.shape[0], 2)), test.to_numpy()[:,2]

### Classification between class1 and class2 
p2 = Perceptron(learning_rate=0.00001, epoch=5000, prediction='regression')
p2.train(trainX, trainY, epoch=True)

predictions_train = p2.predictR(trainX)
predictions_validate = p2.predictR(validateX)
predictions_test = p2.predictR(testX)

def epochVsError_plot(model):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    error = model.avg_error
    nepoch = [i+1 for i in range(len(error))]
    plt.scatter(nepoch, error, marker='o', s=5, facecolors='b', edgecolors='b')
    plt.xlabel('Number of Epoch')
    plt.ylabel('Average Error')
#     plt.title('Average error vs number of epoch for training of Bivariant regression data')
    plt.savefig("AvgErrorVsEpoch_Perceptron_Regression2.png", dpi=600, bbox_inches="tight")
    plt.clf()
    
def modelAndTarget_plot(target, model, label='train'):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    n_item = [i+1 for i in range(len(target))]
    plt.scatter(n_item, model, marker='o', s=5, facecolors='b', edgecolors='b', label='Model Output')
    plt.scatter(n_item, target, marker='s', s=5, facecolors='r', edgecolors='r', label='Target Output')
    plt.legend(bbox_to_anchor=(0.15, 1.2), loc='upper left', ncol=2)
    plt.xlabel('Dataset')
    plt.ylabel('Regression Output')
#     plt.title('Model output and target output of {} dataset'.format(label))
    plt.savefig("modelAndTarget_{}_Perceptron_Regression2.png".format(label), dpi=600, bbox_inches="tight")
    plt.clf()

def modelVsTarget_plot(target, model, label='train'):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    plt.scatter(target, model, marker='o', s=5, facecolors='b', edgecolors='b')
    plt.xlabel('Target Output')
    plt.ylabel('Model Output')
#     plt.title('Model output vs target output of {} dataset'.format(label))
    plt.savefig("modelVsTarget_{}_Perceptron_Regression2.png".format(label), dpi=600, bbox_inches="tight")
    plt.clf()

def MSE(target, model):
    target = np.array(target)
    model = np.array(model)
    mse = np.mean(np.power(target - model, 2))
    return mse

def plot_mse(mse):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    x_data = ['train', 'validation', 'test']
    plt.scatter(x_data, mse, marker='o', s=5, facecolors='b', edgecolors='b')
    plt.xlabel('Type of Dataset')
    plt.ylabel('Mean Square Error')
#     plt.title('Mean squared error (MSE) on training data, validation data and test data')
    plt.savefig("MSE_Perceptron_Regression2.png", dpi=600, bbox_inches="tight")
    plt.clf()
    
### Call the 'epochVsError_plot' function for all pair of classes
epochVsError_plot(p2)
### Model output and target output plot
modelAndTarget_plot(trainY, predictions_train, label='train')
modelAndTarget_plot(testY, predictions_test, label='test')
modelAndTarget_plot(validateY, predictions_validate, label='validation')
### Model output vs target output plot
modelVsTarget_plot(trainY, predictions_train, label='train')
modelVsTarget_plot(testY, predictions_test, label='test')
modelVsTarget_plot(validateY, predictions_validate, label='validation')
### For Mean square Error
mse_train = MSE(trainY, predictions_train)
mse_val = MSE(validateY, predictions_validate)
mse_test = MSE(testY, predictions_test)
plot_mse([mse_train, mse_val, mse_test])


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>